In [1]:
pip install pandas sentence-transformers scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer


# Paso 1: Cargar y preparar los datos de entrenamiento

# Cargar los datos de entrenamiento desde el archivo TRAINING_DATA.txt
data = pd.read_csv('TRAINING_DATA.txt', sep='\t', header=None, names=['tag', 'sentence'])

# Eliminar filas con valores faltantes, si las hay
data.dropna(inplace=True)

# Limpiar el texto de las oraciones
def clean_text(text):
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = text.lower()  # Convert to lowercase
    return text

data['cleaned_text'] = data['sentence'].apply(clean_text)

# Tokenizar el texto de las oraciones (si es necesario)
# Aquí se podría agregar el paso de eliminación de stopwords si se desea

# Paso 2: Entrenar un modelo de embedding de oraciones

# Inicializar el modelo SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Convertir las oraciones a embeddings
X_train_embeddings = model.encode(data['cleaned_text'].tolist())

# Definir las etiquetas
y_train = data['tag']

# Paso 3: Dividir los datos en entrenamiento y prueba

# Dividir los datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_train_embeddings, y_train, test_size=0.2, random_state=42)

# Paso 4: Entrenar un clasificador de Bosques Aleatorios

# Inicializar y entrenar el clasificador Random Forest
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)

# Paso 5: Hacer predicciones en los datos de prueba

# Predecir en los datos de prueba
y_pred = rf_classifier.predict(X_test)

# Evaluar el modelo
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Plotear la matriz de confusión
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2)
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='g', cmap='Blues', cbar=False, annot_kws={'size': 14}, 
            xticklabels=['0', '1'], yticklabels=['0', '1'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

/opt/anaconda3/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-06-12 18:58:16.198230: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


: 